In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)

import warnings
warnings.filterwarnings("ignore")
from thefuzz import fuzz
from thefuzz import process
import datetime

In [2]:
class SizeDataTransformer:
    
    def __init__(self, path):
        self.path = path
        
    def read_data(self):
        if self.path.split(".")[-1] == "xlsx":
            df = pd.read_excel(self.path, None)
            catalog = {}
            for i, j in enumerate(list(df.keys())):
                catalog.setdefault(i + 1, j)
                print(i + 1, j)
            answer = int(input("Please select the sheet"))
            df = df[catalog[answer]]
        else:
            df = pd.read_csv(self.path)
        return df
    
    @staticmethod
    def tire_type(df, category, channel=None, years=True):
        if years == True:
            df = df[(df["KA分類"] == category) & (df["大通路分類"] == channel) & (df["年月"].astype(str).str.contains("2022|2023"))]
        else:
            df = df[(df["KA分類"] == category) & (df["大通路分類"] == channel)]
        df["整理後的日期"] = pd.to_datetime(df["年月"], format="%Y%m")
        return df
        
        
        
    
    

In [3]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()
df = SizeDataTransformer(file_path)
df = df.read_data()

1 工作表1


In [4]:
result = SizeDataTransformer.tire_type(df, "TBR(G)", "REP", False)

In [5]:
result["條數"] = result["條數"].fillna(0)

In [12]:
result["舊分類"] = result["舊分類"].map(lambda x: "LSR" if "LSR" in x else x)

In [13]:
result

,年度,季度,月份,年月,種類,客戶類別,客戶契約,通路明細,客戶代號,客戶名稱,營業所,舊分類,國別,商品代號,規格,條數,Gross,discount,NET金額,大通路分類,單位膠重,總膠重,MTP,預算/前實績,INCH,成本,營業費,花紋,花紋系列,SECTION,GROUP別,KA分類,國內市場財別,其他業外調整,整理後的日期
54,2022,Q1,Jan,202201,外胎,NaN,NaN,FLT-REP,NaN,FLT-REP,本社,LSR,進口,LSR03765,235/75R 175 R294Z 132M,2.0,13400.0,0.0,13400.0,REP,14.351,28.702,LR2,22ACT,175.0,7510.276445,1350.980286,R294,RIB,235/75,REPTBR(G),TBR(G),TBR(G),-7.507428,2022-01-01
59,2022,Q1,Jan,202201,外胎,NaN,NaN,FLT-REP,NaN,FLT-REP,本社,TBR,進口,TBR04720,12R225 152L 16 R150AZ,66.0,599940.0,0.0,599940.0,REP,29.816,1967.856,TR3,22ACT,225.0,631201.387592,60485.605452,R150,RIB,12,REPTBR(G),TBR(G),TBR(G),-336.119887,2022-01-01
60,2022,Q1,Jan,202201,外胎,NaN,NaN,FLT-REP,NaN,FLT-REP,本社,TBR,進口,TBR05258,255/70R225 B40M R294Z,10.0,84080.0,0.0,84080.0,REP,19.100,191.000,TR3,22ACT,225.0,75170.053333,8476.897200,R294,RIB,255/70,REPTBR(G),TBR(G),TBR(G),-47.106311,2022-01-01
61,2022,Q1,Jan,202201,外胎,NaN,NaN,FLT-REP,NaN,FLT-REP,本社,TBR,進口,TBR09920,245/70R17.5 B43J R168Z,2.0,13000.0,0.0,13000.0,REP,15.794,31.588,TR5,22ACT,175.0,8682.965510,1310.652517,R168,RIB,245/70,REPTBR(G),TBR(G),TBR(G),-7.283326,2022-01-01
62,2022,Q1,Jan,202201,外胎,NaN,NaN,FLT-REP,NaN,FLT-REP,本社,TBR,進口,TBR0H350,315/80R225 B54M 18 R150BZ,30.0,240000.0,0.0,240000.0,REP,30.600,918.000,TR3,22ACT,225.0,198202.700624,24196.661847,R150,RIB,315/80,REPTBR(G),TBR(G),TBR(G),-134.461401,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220334,2023,Q1,Feb,202302,外胎,輪胎行,A,輪胎行,N1N1C035,廣益,高雄,TBR,進口,TBR0J600,315/80R225 B54M 18 R150Z,15.0,255000.0,96750.0,158250.0,REP,32.453,486.795,TR3,23ACT,225.0,133917.180577,29058.815788,R150,RIB,315/80,TBR,TBR(G),TBR(G),30.503734,2023-02-01
220335,2023,Q1,Feb,202302,外胎,輪胎行,A,輪胎行,L1L1B087,保晟,高雄,TBR,進口,TBR0J600,315/80R225 B54M 18 R150Z,30.0,510000.0,193477.0,316523.0,REP,32.453,973.590,TR3,23ACT,225.0,267834.361154,58121.854973,R150,RIB,315/80,TBR,TBR(G),TBR(G),61.011901,2023-02-01
220336,2023,Q1,Feb,202302,外胎,輪胎行,A,輪胎行,K1K10035,大榮,高雄,TBR,進口,TBR0J487,315/80R225 B54M M729UZ,15.0,264000.0,99000.0,165000.0,REP,37.279,559.185,TR3,23ACT,225.0,137214.589166,30298.291343,M729,MIX,315/80,TBR,TBR(G),TBR(G),31.804841,2023-02-01
220337,2023,Q1,Feb,202302,外胎,輪胎行,A,輪胎行,L1L1G063,奇盛,高雄,TBR,進口,TBR0J600,315/80R225 B54M 18 R150Z,30.0,510000.0,193477.0,316523.0,REP,32.453,973.590,TR3,23ACT,225.0,267834.361154,58121.854973,R150,RIB,315/80,TBR,TBR(G),TBR(G),61.011901,2023-02-01


In [9]:
result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2023ACT\power_bi_2022-2023_Size Data_All_TBR_rawling_data.xlsx", index=False)

In [ ]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data.xlsx", None)
df.keys()

In [ ]:
df = df["工作表1"]

In [ ]:
df = df[(df["大通路分類"] == "REP") & (df["國內市場財別"] == "TBR(G)")]
df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data_TBR版本.xlsx", index=False)

In [ ]:
df = pd.read_clipboard()

In [ ]:
result = df[(df["Size Designation"].str.startswith("P") == False) & (df["PrCode"].str.contains("TBR|LSR"))]

In [ ]:
bpcom_dict = dict(zip(result["PrCode"], result["Product Name"]))

In [ ]:
new_rule = pd.read_clipboard()

In [ ]:
new_rule_dict = dict(zip(new_rule["Product name"], new_rule["Category"]))

In [ ]:
product_list = new_rule["Product name"].values.tolist()

In [ ]:
isom = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data_TBR版本.xlsx")
isom

In [ ]:
isom["Product Name"] = isom["商品代號"].map(bpcom_dict)

In [ ]:
isom["Product Name"] = isom["Product Name"].fillna("na")

In [ ]:
final_result = pd.concat([isom, pd.DataFrame(isom["Product Name"].map(lambda x: process.extractOne(x, product_list)).tolist(), columns=["匹配商品名稱", "Confidence"])], axis=1)

In [ ]:
final_result["New Category"] = final_result["匹配商品名稱"].map(new_rule_dict)

In [ ]:
final_result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data_TBR版本.xlsx", index=False)

In [2]:
df = pd.read_clipboard()

In [19]:
df.groupby(["month", "company_name", "tyre_type", "category", "upper_tag", "lower_tag"])["en_product_name", "items", "size", "model"].apply(list).reset_index()

,month,company_name,tyre_type,category,upper_tag,lower_tag,0
0,1110106,昆翔國際股份有限公司,輪胎,IC1,8897376,8897785,"[en_product_name, items, size, model]"


In [20]:
df.set_index(["month", "company_name", "tyre_type", "category", "upper_tag", "lower_tag"])[["en_product_name", "items", "size", "model"]].values.tolist()

[['TYRES - RADIAL 315/80R22.5 154/150M M729',
  1,
  '315/80R22.5 154/150M',
  'M729'],
 ['315/80R22.5 154/150M R150', 3, '315/80R22.5 154/150M', 'R150']]